# Import  
파이썬에서 다른 모듈이나 패키지에서 함수, 클래스 등을 가져옵니다.

In [ ]:
!pip install koreanize-matplotlib
import koreanize_matplotlib

In [ ]:
import pandas as pd
import numpy as np
import random
! pip install catboost
from sklearn.preprocessing import RobustScaler, MinMaxScaler,StandardScaler
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import GridSearchCV

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 4.4 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split

#### 시드(seed) 고정  
매번 고정된 결과를 얻기 위해서 사용합니다.  
시드를 고정하지 않는다면 같은 코드라도 매번 다른 결과가 나올 수 있습니다.

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

#### 데이터 읽어오기 및 데이터 확인

In [ ]:
# pd.read_csv() 함수를 사용해서 데이터를 읽어오는 코드입니다.
train = pd.read_csv('train.csv')

# 데이터를 확인하기 위해 head() 함수를 사용합니다.
train.head()

,ID,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지,TARGET
0,TRAIN_00000,9,화요일,10,137,8.0,2.611124,0.000,0.0,0.0,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,2
1,TRAIN_00001,11,화요일,6,438,13.0,3.209093,0.000,0.0,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,0
2,TRAIN_00002,8,일요일,6,1729,47.0,1.619597,0.000,0.0,0.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,인도,1
3,TRAIN_00003,5,월요일,6,2337,53.0,1.921615,11.375,0.0,0.0,225.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,주거지,1
4,TRAIN_00004,9,일요일,11,1439,41.0,1.789721,0.000,0.0,0.0,255.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주유소,2


#### 독립변수(x_train), 종속변수(y_train)로 분리하기  


In [ ]:
x_train = train.drop(['ID', 'TARGET'], axis = 1)
y_train = train['TARGET']


In [ ]:
x_train.head()

,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지
0,9,화요일,10,137,8.0,2.611124,0.000,0.0,0.0,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도
1,11,화요일,6,438,13.0,3.209093,0.000,0.0,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도
2,8,일요일,6,1729,47.0,1.619597,0.000,0.0,0.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,인도
3,5,월요일,6,2337,53.0,1.921615,11.375,0.0,0.0,225.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,주거지
4,9,일요일,11,1439,41.0,1.789721,0.000,0.0,0.0,255.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주유소


#### train , validation data 나누기

In [ ]:
X_tr, X_val, Y_tr, Y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

#### 라벨인코딩(Label Encoding)

In [ ]:

ordinal_features = ['요일', '범죄발생지']

for feature in ordinal_features:
    le = LabelEncoder()

    # Fit and transform on the training data
    le = le.fit(X_tr[feature])
    X_tr[feature] = le.transform(X_tr[feature])

    # Transform the test data (Note: Only transform, do not fit again to avoid data leakage)
    X_val[feature] = le.transform(X_val[feature])



#### 원핫인코딩

### catboostClassifier

In [ ]:
cat_model = CatBoostClassifier(random_state=42,iterations=250,learning_rate=0.1)
cat_model.fit(X_tr, Y_tr)

y_pred = cat_model.predict(X_val)


0:	learn: 1.0789886	total: 97.5ms	remaining: 24.3s
1:	learn: 1.0634531	total: 162ms	remaining: 20s
2:	learn: 1.0499585	total: 203ms	remaining: 16.7s
3:	learn: 1.0392529	total: 265ms	remaining: 16.3s
4:	learn: 1.0305634	total: 329ms	remaining: 16.1s
5:	learn: 1.0233867	total: 400ms	remaining: 16.3s
6:	learn: 1.0175835	total: 468ms	remaining: 16.3s
7:	learn: 1.0131383	total: 512ms	remaining: 15.5s
8:	learn: 1.0084699	total: 594ms	remaining: 15.9s
9:	learn: 1.0046551	total: 714ms	remaining: 17.1s
10:	learn: 1.0017625	total: 805ms	remaining: 17.5s
11:	learn: 0.9987128	total: 898ms	remaining: 17.8s
12:	learn: 0.9970875	total: 958ms	remaining: 17.5s
13:	learn: 0.9947349	total: 1.02s	remaining: 17.3s
14:	learn: 0.9931455	total: 1.09s	remaining: 17.1s
15:	learn: 0.9918380	total: 1.16s	remaining: 17s
16:	learn: 0.9903155	total: 1.25s	remaining: 17.2s
17:	learn: 0.9893854	total: 1.33s	remaining: 17.1s
18:	learn: 0.9880194	total: 1.39s	remaining: 16.9s
19:	learn: 0.9870105	total: 1.47s	remaining:

#### 예측

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

acc=accuracy_score(Y_val,y_pred)
print(f'정확도 : {acc:.4f}')

# 정밀도 (precision)
precision=precision_score(Y_val,y_pred, average='weighted')
print(f'정말도 : {precision:.4f}')

recall = recall_score(Y_val, y_pred, average='weighted')
print(f'Recall: {recall:.4f}')

# F1 점수 (F1 Score)
f1 = f1_score(Y_val, y_pred, average='weighted')
print(f'F1 Score: {f1:.4f}')

# 혼동 행렬 (Confusion Matrix)
conf_matrix = confusion_matrix(Y_val, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

정확도 : 0.5519
정말도 : 0.5497
Recall: 0.5519
F1 Score: 0.5413
Confusion Matrix:
[[8021 1257 1641]
 [3040 3293 1309]
 [3084 1016 2661]]


In [ ]:
from sklearn.feature_selection import RFE

In [ ]:
for n_features in range(1, 17):
    # RFE를 사용하여 특성 선택
    rfe = RFE(estimator=cat_model, n_features_to_select=n_features)
    X_train_rfe = rfe.fit_transform(X_tr, Y_tr)
    X_test_rfe = rfe.transform(X_val)

    # 모델 훈련 및 예측
    cat_model.fit(X_train_rfe, Y_tr)
    y_pred_rf = cat_model.predict(X_test_rfe)

    # 정확도 출력
    accuracy_rf = accuracy_score(Y_val, y_pred_rf)
    print(f'n_features_to_select={n_features}, Accuracy: {accuracy_rf:.2f}')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
4:	learn: 1.0309185	total: 188ms	remaining: 9.23s
5:	learn: 1.0236463	total: 217ms	remaining: 8.83s
6:	learn: 1.0173613	total: 247ms	remaining: 8.58s
7:	learn: 1.0119843	total: 274ms	remaining: 8.3s
8:	learn: 1.0077892	total: 304ms	remaining: 8.15s
9:	learn: 1.0039842	total: 347ms	remaining: 8.34s
10:	learn: 1.0009221	total: 381ms	remaining: 8.27s
11:	learn: 0.9982015	total: 411ms	remaining: 8.15s
12:	learn: 0.9954546	total: 439ms	remaining: 8.01s
13:	learn: 0.9935003	total: 491ms	remaining: 8.28s
14:	learn: 0.9917462	total: 531ms	remaining: 8.32s
15:	learn: 0.9902991	total: 561ms	remaining: 8.2s
16:	learn: 0.9892206	total: 589ms	remaining: 8.07s
17:	learn: 0.9875447	total: 616ms	remaining: 7.93s
18:	learn: 0.9863443	total: 643ms	remaining: 7.82s
19:	learn: 0.9852503	total: 673ms	remaining: 7.73s
20:	learn: 0.9844409	total: 701ms	remaining: 7.65s
21:	learn: 0.9835305	total: 726ms	remaining: 7.53s
22:	learn: 0.9824378	total: 763ms	remaining: 7.53s
23:

### XGBoost